In [2]:
import findspark
findspark.init("/home/raj/spark/")

In [5]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.classification import LinearSVC
from pyspark.ml.feature import VectorAssembler

def Linear_svm():
    

    spark = SparkSession\
        .builder\
        .appName("Python Spark ML example")\
        .getOrCreate()
# Loads data.
    dataset= spark\
    .read\
    .option("inferSchema", "true")\
    .option("header", "true")\
    .csv("/home/raj/Downloads/notenook/adult2.csv")
    dataset.toPandas()

#creating feature vectors
    workclass_stringIndexer = StringIndexer(inputCol="workclass", outputCol="workclass_indexed")
    si_model_workclass = workclass_stringIndexer.fit(dataset)
    education_stringIndexer = StringIndexer(inputCol="education", outputCol="education_indexed")
    si_model_education = education_stringIndexer.fit(dataset)
    occupation_stringIndexer = StringIndexer(inputCol="occupation", outputCol="occupation_indexed")
    si_model_occupation = occupation_stringIndexer.fit(dataset)
    sex_stringIndexer = StringIndexer(inputCol="sex", outputCol="sex_indexed")
    si_model_sex = sex_stringIndexer.fit(dataset)
    income_stringIndexer = StringIndexer(inputCol="income", outputCol="income_indexed")
    si_model_income = income_stringIndexer.fit(dataset)
    td = si_model_workclass.transform(si_model_education.transform(si_model_occupation.transform(si_model_sex.transform(si_model_income.transform(dataset)))))
    td.toPandas()


#feature_columns = ["age","workclass_indexed","fnlwgt","education_num","hours_per_week","sex_indexed","education_indexed","occupation_indexed","income_indexed"]
    feature_columns = ["age","sex_indexed"]
    feature_assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

    td_dataset_prepared = feature_assembler.transform(td)
    td_dataset_prepared.printSchema()
    td_dataset_prepared.toPandas()

    (trainingData, testData) = td_dataset_prepared.randomSplit([0.8,0.2])


#initialises the svmmodel
    svm = LinearSVC(maxIter=5, regParam=0.04,labelCol="income_indexed")

#trainms the svm with training data
    model = svm.fit(trainingData)

#provides coefficients for w and b
    model.coefficients

    model.numFeatures


    model.numClasses

#tests on a test dataset
    
    
    return model.transform(testData).toPandas()



In [6]:
Linear_svm()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: double (nullable = true)
 |-- education: string (nullable = true)
 |-- education_num: double (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital_gain: double (nullable = true)
 |-- capital_loss: double (nullable = true)
 |-- hours_per_week: double (nullable = true)
 |-- native_country: string (nullable = true)
 |-- income: string (nullable = true)
 |-- income_indexed: double (nullable = true)
 |-- sex_indexed: double (nullable = true)
 |-- occupation_indexed: double (nullable = true)
 |-- education_indexed: double (nullable = true)
 |-- workclass_indexed: double (nullable = true)
 |-- features: vector (nullable = true)



,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,...,native_country,income,income_indexed,sex_indexed,occupation_indexed,education_indexed,workclass_indexed,features,rawPrediction,prediction
0,17,?,89870.0,10th,6.0,Never-married,?,Own-child,White,Male,...,United-States,<=50K,0.0,0.0,7.0,7.0,3.0,"[17.0, 0.0]","[0.5271201425346791, -0.5271201425346791]",0.0
1,17,?,158762.0,10th,6.0,Never-married,?,Own-child,White,Female,...,United-States,<=50K,0.0,1.0,7.0,7.0,3.0,"[17.0, 1.0]","[0.8749572749729366, -0.8749572749729366]",0.0
2,17,?,202521.0,11th,7.0,Never-married,?,Own-child,White,Male,...,United-States,<=50K,0.0,0.0,7.0,5.0,3.0,"[17.0, 0.0]","[0.5271201425346791, -0.5271201425346791]",0.0
3,17,Local-gov,32124.0,9th,5.0,Never-married,Other-service,Own-child,Black,Male,...,United-States,<=50K,0.0,0.0,4.0,8.0,2.0,"[17.0, 0.0]","[0.5271201425346791, -0.5271201425346791]",0.0
4,17,Private,32607.0,10th,6.0,Never-married,Farming-fishing,Own-child,White,Male,...,United-States,<=50K,0.0,0.0,11.0,7.0,0.0,"[17.0, 0.0]","[0.5271201425346791, -0.5271201425346791]",0.0
5,17,Private,122041.0,11th,7.0,Never-married,Craft-repair,Own-child,White,Male,...,United-States,<=50K,0.0,0.0,2.0,5.0,0.0,"[17.0, 0.0]","[0.5271201425346791, -0.5271201425346791]",0.0
6,17,Private,239947.0,11th,7.0,Never-married,Adm-clerical,Own-child,White,Male,...,United-States,<=50K,0.0,0.0,5.0,5.0,0.0,"[17.0, 0.0]","[0.5271201425346791, -0.5271201425346791]",0.0
7,17,Private,241185.0,12th,8.0,Never-married,Prof-specialty,Own-child,White,Male,...,United-States,<=50K,0.0,0.0,0.0,12.0,0.0,"[17.0, 0.0]","[0.5271201425346791, -0.5271201425346791]",0.0
8,17,Private,242718.0,11th,7.0,Never-married,Sales,Own-child,White,Male,...,United-States,<=50K,0.0,0.0,3.0,5.0,0.0,"[17.0, 0.0]","[0.5271201425346791, -0.5271201425346791]",0.0
9,18,?,165532.0,12th,8.0,Never-married,?,Own-child,White,Male,...,United-States,<=50K,0.0,0.0,7.0,12.0,3.0,"[18.0, 0.0]","[0.546137334584208, -0.546137334584208]",0.0
